In [3]:
import magellan as mg

In [4]:
import pandas as pd

In [5]:
mg.init_jvm()

True

In [6]:
# read tables
A = mg.read_csv('tableA.csv')
B = mg.read_csv('tableB.csv')

In [7]:
# Blocking - already done and stored in tableC.csv
C = mg.read_csv('Name-Address-CandSet.csv', ltable=A, rtable=B)
len(C)

1601

In [8]:
# Sample the candidate set
S = mg.sample_table(C, 450)

In [9]:
# For the demo, use pre-labeled data
G = mg.read_csv('labeled-data-v4.csv', ltable=A, rtable=B)
len(G)

448

In [10]:
# Split G into development (I) and evaluation (J)
IJ = mg.train_test_split(G, train_proportion=0.7)
I = IJ['train']
J = IJ['test']

In [11]:
# check the number of tuples in I and J
(len(I), len(J))

(313, 135)

In [13]:
# Selecting the best learning-based matcher using I

# Plan 

# 1. Create a set of ML-matchers
# 2. Generate features --> feature_table
# 3. Extract feature vectors using I and feature_table
# 4. Select best learning-based matcher using CV
# 5. Debug the selected matcher (and repeat the above steps)

In [15]:
#Store Development Set
I.to_csv('DevelopmentSet.csv')
#Store Evaluation Set
J.to_csv('EvaluationSet.csv')

True

In [16]:
# Create a set of ML-matchers
dt = mg.DTMatcher()
svm = mg.SVMMatcher()
rf = mg.RFMatcher(name='RF', random_state=0)
nb = mg.NBMatcher(name='NB')
lg = mg.LogRegMatcher(name='LogReg')
ln = mg.LinRegMatcher(name='LinReg')

In [17]:
# Generate features
feat_table = mg.get_features_for_matching(A, B)

In [20]:
len(feat_table)

32

In [22]:
feat_table

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source
0,_id__id_exm,_id,_id,None,None,exact_match,<function _id__id_exm at 0x7eff1d6759b0>,from magellan.feature.simfunctions import *\nf...
1,_id__id_anm,_id,_id,None,None,abs_norm,<function _id__id_anm at 0x7eff1d675de8>,from magellan.feature.simfunctions import *\nf...
2,_id__id_lev,_id,_id,None,None,lev,<function _id__id_lev at 0x7eff1d675938>,from magellan.feature.simfunctions import *\nf...
3,ID_ID_exm,ID,ID,None,None,exact_match,<function ID_ID_exm at 0x7eff1d675d70>,from magellan.feature.simfunctions import *\nf...
4,ID_ID_anm,ID,ID,None,None,abs_norm,<function ID_ID_anm at 0x7eff1d675e60>,from magellan.feature.simfunctions import *\nf...
5,ID_ID_lev,ID,ID,None,None,lev,<function ID_ID_lev at 0x7eff1d675ed8>,from magellan.feature.simfunctions import *\nf...
6,NAME_NAME_jac_qgm_3_qgm_3,NAME,NAME,qgm_3,qgm_3,jaccard,<function NAME_NAME_jac_qgm_3_qgm_3 at 0x7eff1...,from magellan.feature.simfunctions import *\nf...
7,NAME_NAME_cos_dlm_dc0_dlm_dc0,NAME,NAME,dlm_dc0,dlm_dc0,cosine,<function NAME_NAME_cos_dlm_dc0_dlm_dc0 at 0x7...,from magellan.feature.simfunctions import *\nf...
8,NAME_NAME_jac_dlm_dc0_dlm_dc0,NAME,NAME,dlm_dc0,dlm_dc0,jaccard,<function NAME_NAME_jac_dlm_dc0_dlm_dc0 at 0x7...,from magellan.feature.simfunctions import *\nf...
9,NAME_NAME_mel,NAME,NAME,None,None,monge_elkan,<function NAME_NAME_mel at 0x7eff03a0e140>,from magellan.feature.simfunctions import *\nf...


In [23]:
feat_table.to_csv("FeatureSetInApproach1.csv")

In [15]:

#feat_subset_iter1 = feat_table[6:9]

In [16]:
#feat_subset_iter5 = feat_table[10:11]

In [17]:
#feat_subset_iter2 = feat_table[20:25]

In [18]:
#feat_subset_iter3 = feat_table[28:30]

In [19]:
#feat_subset_iter4 = feat_table[31:]

In [20]:
#feat_subset_concat = pd.concat([feat_subset_iter1, feat_subset_iter5, feat_subset_iter2, feat_subset_iter3,feat_subset_iter4])

In [63]:
#feat_subset_concat

In [24]:
# Get feature vectors
K = mg.extract_feature_vecs(I, feature_table=feat_table, attrs_after='gold')

In [25]:
K.head()

,_id,ltable._id,rtable._id,_id__id_exm,_id__id_anm,_id__id_lev,ID_ID_exm,ID_ID_anm,ID_ID_lev,NAME_NAME_jac_qgm_3_qgm_3,...,PHONENUMBER_PHONENUMBER_sw,PHONENUMBER_PHONENUMBER_swg,NO_OF_REVIEWS_NO_OF_REVIEWS_exm,NO_OF_REVIEWS_NO_OF_REVIEWS_anm,NO_OF_REVIEWS_NO_OF_REVIEWS_lev,ADDRESS_ADDRESS_jac_qgm_3_qgm_3,ADDRESS_ADDRESS_cos_dlm_dc0_dlm_dc0,ADDRESS_ADDRESS_mel,ADDRESS_ADDRESS_lev,gold
0,0,1494,1143,0,0.765060,0.25,0,0.997228,0.615385,0.000000,...,0.178571,0.142857,0,0.273504,0.000000,0.485714,0.668153,0.815385,0.718750,0
1,1,2878,5634,0,0.510827,0.00,0,0.997228,0.384615,1.000000,...,1.000000,1.000000,0,0.253659,0.000000,0.833333,0.912871,1.000000,0.842105,1
2,2,1252,3945,0,0.317364,0.00,0,0.997228,0.384615,0.066667,...,0.428571,0.428571,0,0.035088,0.333333,0.612903,0.730297,0.880000,0.709677,0
3,3,2745,5305,0,0.517436,0.25,0,0.997228,0.384615,1.000000,...,1.000000,1.000000,0,0.380607,0.250000,0.717949,0.771517,0.939394,0.815789,1
4,4,39,4679,0,0.008335,0.25,0,0.997228,0.384615,0.241379,...,0.571429,0.600000,0,0.297386,0.000000,0.450000,0.577350,0.740741,0.628571,0


In [26]:
# impute K
K.fillna(0, inplace=True)

In [27]:
K.head()

,_id,ltable._id,rtable._id,_id__id_exm,_id__id_anm,_id__id_lev,ID_ID_exm,ID_ID_anm,ID_ID_lev,NAME_NAME_jac_qgm_3_qgm_3,...,PHONENUMBER_PHONENUMBER_sw,PHONENUMBER_PHONENUMBER_swg,NO_OF_REVIEWS_NO_OF_REVIEWS_exm,NO_OF_REVIEWS_NO_OF_REVIEWS_anm,NO_OF_REVIEWS_NO_OF_REVIEWS_lev,ADDRESS_ADDRESS_jac_qgm_3_qgm_3,ADDRESS_ADDRESS_cos_dlm_dc0_dlm_dc0,ADDRESS_ADDRESS_mel,ADDRESS_ADDRESS_lev,gold
0,0,1494,1143,0,0.765060,0.25,0,0.997228,0.615385,0.000000,...,0.178571,0.142857,0,0.273504,0.000000,0.485714,0.668153,0.815385,0.718750,0
1,1,2878,5634,0,0.510827,0.00,0,0.997228,0.384615,1.000000,...,1.000000,1.000000,0,0.253659,0.000000,0.833333,0.912871,1.000000,0.842105,1
2,2,1252,3945,0,0.317364,0.00,0,0.997228,0.384615,0.066667,...,0.428571,0.428571,0,0.035088,0.333333,0.612903,0.730297,0.880000,0.709677,0
3,3,2745,5305,0,0.517436,0.25,0,0.997228,0.384615,1.000000,...,1.000000,1.000000,0,0.380607,0.250000,0.717949,0.771517,0.939394,0.815789,1
4,4,39,4679,0,0.008335,0.25,0,0.997228,0.384615,0.241379,...,0.571429,0.600000,0,0.297386,0.000000,0.450000,0.577350,0.740741,0.628571,0


In [31]:
# select the best ML matcher using CV
result = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=K, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='precision', random_state=0) 

In [32]:
result['selected_matcher']

In [33]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree_82108063030395210541,<magellan.matcher.dtmatcher.DTMatcher object a...,5,1,0.916667,1.000000,1,1,0.983333
1,RF,<magellan.matcher.rfmatcher.RFMatcher object a...,5,1,1.000000,1.000000,1,1,1.000000
2,SVM_04604073030395210541,<magellan.matcher.svmmatcher.SVMMatcher object...,5,1,1.000000,1.000000,1,0,0.800000
3,NB,<magellan.matcher.nbmatcher.NBMatcher object a...,5,1,0.954545,0.928571,1,1,0.976623
4,LogReg,<magellan.matcher.logregmatcher.LogRegMatcher ...,5,1,1.000000,0.928571,1,1,0.985714
5,LinReg,<magellan.matcher.linregmatcher.LinRegMatcher ...,5,1,0.954545,1.000000,1,1,0.990909


In [34]:
# select the best ML matcher using CV
result = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=K, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='recall', random_state=0) 

In [35]:
result['selected_matcher']

In [36]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree_82108063030395210541,<magellan.matcher.dtmatcher.DTMatcher object a...,5,0.933333,1.000000,1.000000,1.000000,1.000000,0.986667
1,RF,<magellan.matcher.rfmatcher.RFMatcher object a...,5,0.933333,1.000000,1.000000,0.947368,1.000000,0.976140
2,SVM_04604073030395210541,<magellan.matcher.svmmatcher.SVMMatcher object...,5,0.133333,0.136364,0.076923,0.105263,0.000000,0.090377
3,NB,<magellan.matcher.nbmatcher.NBMatcher object a...,5,1.000000,0.954545,1.000000,0.947368,1.000000,0.980383
4,LogReg,<magellan.matcher.logregmatcher.LogRegMatcher ...,5,0.933333,1.000000,1.000000,0.947368,0.928571,0.961855
5,LinReg,<magellan.matcher.linregmatcher.LinRegMatcher ...,5,0.933333,0.954545,0.846154,0.894737,1.000000,0.925754


In [39]:
# select the best ML matcher using CV
result = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=K, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='f1', random_state=0) 

In [40]:
result['selected_matcher']

In [41]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree_82108063030395210541,<magellan.matcher.dtmatcher.DTMatcher object a...,5,0.965517,0.956522,1.000000,0.972973,1.000000,0.979002
1,RF,<magellan.matcher.rfmatcher.RFMatcher object a...,5,0.965517,1.000000,1.000000,0.972973,1.000000,0.987698
2,SVM_04604073030395210541,<magellan.matcher.svmmatcher.SVMMatcher object...,5,0.235294,0.240000,0.142857,0.190476,0.000000,0.161725
3,NB,<magellan.matcher.nbmatcher.NBMatcher object a...,5,1.000000,0.954545,0.962963,0.972973,1.000000,0.978096
4,LogReg,<magellan.matcher.logregmatcher.LogRegMatcher ...,5,0.965517,1.000000,0.962963,0.972973,0.962963,0.972883
5,LinReg,<magellan.matcher.linregmatcher.LinRegMatcher ...,5,0.965517,0.954545,0.916667,0.944444,1.000000,0.956235


In [42]:
I_test = mg.cv_matcher_and_trigger(rf, [], table = K, 
                                   exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
                                  target_attr='gold', k=10)
I_test['cv_stats']

0%      100%
[##########] | ETA[sec]: 0.000 
Total time elapsed: 0.351 sec


,Metric,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Fold 6,Fold 7,Fold 8,Fold 9,Fold 10,Mean score
0,precision,10,1.000000,1,1,1,1,1,1,1,1,0.888889,0.988889
1,recall,10,0.900000,1,1,1,1,1,1,1,1,1.000000,0.990000
2,f1,10,0.947368,1,1,1,1,1,1,1,1,0.941176,0.988854


In [117]:
# Split feature vectors to train and test
UV = mg.train_test_split(K, train_proportion=0.7, random_state=0)
U = UV['train']
V = UV['test']

In [118]:
# Train ln using U
rf.fit(table=U, 
       exclude_attrs=['_id', 'ltable._id', 'rtable._id', 'gold'], 
       target_attr='gold')

L = rf.predict(table=V, exclude_attrs=['_id', 'ltable._id', 'rtable._id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)
eval_result = mg.eval_matches(L, 'gold', 'predicted')
mg.print_eval_summary(eval_result)

Precision : 100.0% (25/25)
Recall : 96.15% (25/26)
F1 : 98.04%
False positives : 0 (out of 25 positive predictions)
False negatives : 1 (out of 69 negative predictions)


In [119]:
# Invoke debug interface to check FP and FN
mg.vis_debug_rf(rf, U, V, 
        exclude_attrs=['_id', 'ltable._id', 'rtable._id', 'gold'],
        target_attr='gold')

In [47]:
feat_table

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source
0,_id__id_exm,_id,_id,None,None,exact_match,<function _id__id_exm at 0x7eff1d6759b0>,from magellan.feature.simfunctions import *\nf...
1,_id__id_anm,_id,_id,None,None,abs_norm,<function _id__id_anm at 0x7eff1d675de8>,from magellan.feature.simfunctions import *\nf...
2,_id__id_lev,_id,_id,None,None,lev,<function _id__id_lev at 0x7eff1d675938>,from magellan.feature.simfunctions import *\nf...
3,ID_ID_exm,ID,ID,None,None,exact_match,<function ID_ID_exm at 0x7eff1d675d70>,from magellan.feature.simfunctions import *\nf...
4,ID_ID_anm,ID,ID,None,None,abs_norm,<function ID_ID_anm at 0x7eff1d675e60>,from magellan.feature.simfunctions import *\nf...
5,ID_ID_lev,ID,ID,None,None,lev,<function ID_ID_lev at 0x7eff1d675ed8>,from magellan.feature.simfunctions import *\nf...
6,NAME_NAME_jac_qgm_3_qgm_3,NAME,NAME,qgm_3,qgm_3,jaccard,<function NAME_NAME_jac_qgm_3_qgm_3 at 0x7eff1...,from magellan.feature.simfunctions import *\nf...
7,NAME_NAME_cos_dlm_dc0_dlm_dc0,NAME,NAME,dlm_dc0,dlm_dc0,cosine,<function NAME_NAME_cos_dlm_dc0_dlm_dc0 at 0x7...,from magellan.feature.simfunctions import *\nf...
8,NAME_NAME_jac_dlm_dc0_dlm_dc0,NAME,NAME,dlm_dc0,dlm_dc0,jaccard,<function NAME_NAME_jac_dlm_dc0_dlm_dc0 at 0x7...,from magellan.feature.simfunctions import *\nf...
9,NAME_NAME_mel,NAME,NAME,None,None,monge_elkan,<function NAME_NAME_mel at 0x7eff03a0e140>,from magellan.feature.simfunctions import *\nf...


In [48]:
len(feat_table)

32

In [51]:
#Add trigger - target false negative: use title related feature
neg_trigger = mg.MatchTrigger()
#neg_trigger_phone.add_cond_rule('PHONENUMBER_PHONENUMBER_jac_qgm_3_qgm_3(ltuple, rtuple) != 1.0',feat_table)
#neg_trigger.add_cond_rule(['PHONENUMBER_PHONENUMBER_jac_qgm_3_qgm_3(ltuple, rtuple) != 1.0','NAME_NAME_mel(ltuple, rtuple) < 0.8','ADDRESS_ADDRESS_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.1'],feat_table)
neg_trigger.add_cond_rule('NAME_NAME_mel(ltuple, rtuple) > 0.85',feat_table)
neg_trigger.add_cond_rule('ADDRESS_ADDRESS_mel(ltuple, rtuple) > 0.9',feat_table)
#neg_trigger.add_cond_rule('ADDRESS_ADDRESS_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.5',feat_table)
#neg_trigger_phone.add_cond_rule('NAME_NAME_jac_qgm_3_qgm_3(ltuple,rtuple)!=1.0',feat_table)
neg_trigger.add_cond_status(False)
neg_trigger.add_action(0)

True

In [68]:
#neg_trigger_name = mg.MatchTrigger()
#neg_trigger.add_cond_rule(['PHONENUMBER_PHONENUMBER_jac_qgm_3_qgm_3(ltuple, rtuple) != 1.0','NAME_NAME_mel(ltuple, rtuple) < 0.8','ADDRESS_ADDRESS_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.1'],feat_table)
#neg_trigger_name.add_cond_rule('NAME_NAME_jac_qgm_3_qgm_3(ltuple, rtuple)>0.9',feat_table)
#neg_trigger.add_cond_rule('ADDRESS_ADDRESS_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.5',feat_table)
#neg_trigger_name.add_cond_status(True)
#neg_trigger_name.add_action(0)

True

In [86]:
#neg_trigger_address = mg.MatchTrigger()
#neg_trigger.add_cond_rule(['PHONENUMBER_PHONENUMBER_jac_qgm_3_qgm_3(ltuple, rtuple) != 1.0','NAME_NAME_mel(ltuple, rtuple) < 0.8','ADDRESS_ADDRESS_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.1'],feat_table)
#neg_trigger_address.add_cond_rule('NAME_NAME_lev(ltuple, rtuple) > 0.85',feat_table)
#neg_trigger_address.add_cond_rule('ADDRESS_ADDRESS_mel(ltuple, rtuple) < 0.9',feat_table)
#neg_trigger_address.add_cond_status(True)
#neg_trigger_address.add_action(0)

True

In [52]:
# Apply trigger
Q = neg_trigger.execute(L, 'predicted', inplace=False)
#S = neg_trigger_address.execute(Q,'predicted', inplace = False)
#R = neg_trigger_name.execute(Q, 'predicted', inplace=False)

In [53]:
# Evaluate the result
eval_result = mg.eval_matches(Q, 'gold', 'predicted')
mg.print_eval_summary(eval_result)

Precision : 100.0% (23/23)
Recall : 88.46% (23/26)
F1 : 93.88%
False positives : 0 (out of 23 positive predictions)
False negatives : 3 (out of 71 negative predictions)


In [123]:
Q.to_csv("Result_After_Trigger.csv")

True

In [111]:
I_test = mg.cv_matcher_and_trigger(rf, [neg_trigger], table = K, 
                                   exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
                                  target_attr='gold', k=10)
I_test['cv_stats']

0%      100%
[##########] | ETA[sec]: 0.000 
Total time elapsed: 0.602 sec


,Metric,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Fold 6,Fold 7,Fold 8,Fold 9,Fold 10,Mean score
0,precision,10,1,1.000000,1,1,1,1,1,1.000000,1.000000,1,1.000000
1,recall,10,1,0.916667,1,1,1,1,1,0.666667,0.857143,1,0.944048
2,f1,10,1,0.956522,1,1,1,1,1,0.800000,0.923077,1,0.967960


In [56]:
# Split feature vectors to train and test
UV = mg.train_test_split(K, train_proportion=0.5, random_state=0)
U = UV['train']
V = UV['test']

In [57]:
# Train ln using U
rf.fit(table=U, 
       exclude_attrs=['_id', 'ltable._id', 'rtable._id', 'gold'], 
       target_attr='gold')

L = rf.predict(table=V, exclude_attrs=['_id', 'ltable._id', 'rtable._id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)
eval_result = mg.eval_matches(L, 'gold', 'predicted')
mg.print_eval_summary(eval_result)

Precision : 100.0% (44/44)
Recall : 97.78% (44/45)
F1 : 98.88%
False positives : 0 (out of 44 positive predictions)
False negatives : 1 (out of 113 negative predictions)


In [58]:
# Invoke debug interface to check FP and FN
mg.vis_debug_rf(rf, U, V, 
        exclude_attrs=['_id', 'ltable._id', 'rtable._id', 'gold'],
        target_attr='gold')

In [60]:
feat_subset_iter1 = feat_table[6:9]
feat_subset_iter5 = feat_table[10:11]
feat_subset_iter2 = feat_table[20:25]
feat_subset_iter3 = feat_table[28:30]
feat_subset_iter4 = feat_table[31:]
feat_subset_concat = pd.concat([feat_subset_iter1, feat_subset_iter5, feat_subset_iter2, feat_subset_iter3,feat_subset_iter4])
len(feat_subset_concat)

12

In [62]:
#Write new Feature Set to csv file
feat_subset_concat.to_csv('NewFeatureSet.csv')

In [75]:
# Get feature vectors
K_feat = mg.extract_feature_vecs(I, feature_table=feat_subset_concat, attrs_after='gold')
K_feat.fillna(0, inplace=True)

result = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=K_feat, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='precision', random_state=0) 
result['selected_matcher']

In [76]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree_82108063030395210541,<magellan.matcher.dtmatcher.DTMatcher object a...,5,1,0.956522,0.923077,1.00,1,0.975920
1,RF,<magellan.matcher.rfmatcher.RFMatcher object a...,5,1,0.916667,0.928571,1.00,1,0.969048
2,SVM_04604073030395210541,<magellan.matcher.svmmatcher.SVMMatcher object...,5,1,1.000000,1.000000,1.00,0,0.800000
3,NB,<magellan.matcher.nbmatcher.NBMatcher object a...,5,1,0.916667,0.928571,0.95,1,0.959048
4,LogReg,<magellan.matcher.logregmatcher.LogRegMatcher ...,5,1,1.000000,0.928571,1.00,1,0.985714
5,LinReg,<magellan.matcher.linregmatcher.LinRegMatcher ...,5,1,0.916667,0.866667,1.00,1,0.956667


In [77]:
result = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=K_feat, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='recall', random_state=0) 
result['selected_matcher']

In [78]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree_82108063030395210541,<magellan.matcher.dtmatcher.DTMatcher object a...,5,1.000000,0.954545,0.923077,0.947368,1.000000,0.964998
1,RF,<magellan.matcher.rfmatcher.RFMatcher object a...,5,0.933333,1.000000,1.000000,0.894737,1.000000,0.965614
2,SVM_04604073030395210541,<magellan.matcher.svmmatcher.SVMMatcher object...,5,0.133333,0.090909,0.076923,0.052632,0.000000,0.070759
3,NB,<magellan.matcher.nbmatcher.NBMatcher object a...,5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,LogReg,<magellan.matcher.logregmatcher.LogRegMatcher ...,5,0.933333,0.909091,1.000000,0.894737,0.785714,0.904575
5,LinReg,<magellan.matcher.linregmatcher.LinRegMatcher ...,5,1.000000,1.000000,1.000000,0.947368,0.928571,0.975188


In [79]:
result = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=K_feat, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='f1', random_state=0) 
result['selected_matcher']

In [81]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree_82108063030395210541,<magellan.matcher.dtmatcher.DTMatcher object a...,5,1.000000,0.933333,0.923077,0.972973,1.000000,0.965877
1,RF,<magellan.matcher.rfmatcher.RFMatcher object a...,5,0.965517,0.956522,0.962963,0.944444,1.000000,0.965889
2,SVM_04604073030395210541,<magellan.matcher.svmmatcher.SVMMatcher object...,5,0.235294,0.166667,0.142857,0.100000,0.000000,0.128964
3,NB,<magellan.matcher.nbmatcher.NBMatcher object a...,5,1.000000,0.956522,0.962963,0.974359,1.000000,0.978769
4,LogReg,<magellan.matcher.logregmatcher.LogRegMatcher ...,5,0.965517,0.952381,0.962963,0.944444,0.880000,0.941061
5,LinReg,<magellan.matcher.linregmatcher.LinRegMatcher ...,5,1.000000,0.956522,0.928571,0.972973,0.962963,0.964206


In [82]:
# Split feature vectors to train and test
UV = mg.train_test_split(K_feat, train_proportion=0.7, random_state=0)
U = UV['train']
V = UV['test']

In [83]:
# Train ln using U
nb.fit(table=U, 
       exclude_attrs=['_id', 'ltable._id', 'rtable._id', 'gold'], 
       target_attr='gold')

L = nb.predict(table=V, exclude_attrs=['_id', 'ltable._id', 'rtable._id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)
eval_result = mg.eval_matches(L, 'gold', 'predicted')
mg.print_eval_summary(eval_result)

Precision : 100.0% (26/26)
Recall : 100.0% (26/26)
F1 : 100.0%
False positives : 0 (out of 26 positive predictions)
False negatives : 0 (out of 68 negative predictions)


In [129]:
I_test = mg.cv_matcher_and_trigger(nb, [], table = K_feat, 
                                   exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
                                  target_attr='gold', k=10)
I_test['cv_stats']

0%      100%
[##########] | ETA[sec]: 0.000 
Total time elapsed: 0.190 sec


,Metric,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Fold 6,Fold 7,Fold 8,Fold 9,Fold 10,Mean score
0,precision,10,1,1,1,1,0.909091,1,1,0.857143,1,0.900000,0.966623
1,recall,10,1,1,1,1,1.000000,1,1,0.857143,1,1.000000,0.985714
2,f1,10,1,1,1,1,0.952381,1,1,0.857143,1,0.947368,0.975689


In [94]:
# Now Z is DT (features: feat_subset_iter2) + neg_trigger
# Validate Z using J
# Steps
# 1. Extract feature vectors (using feat_subset_iter2) -- > M
# 2. Train DT using H (feature vectors generated using I)
# 3. Predict M using DT
# 4. Apply negative trigger
# 5. Evaluate the result

In [103]:
#Test using Approach 1
# Extract feature vectors
M = mg.extract_feature_vecs(J, feature_table=feat_table, attrs_after='gold')

In [104]:
# Impute missing values
M.fillna(0, inplace=True)

In [105]:
# Train using feature vectors from I 
rf.fit(table=K, 
       exclude_attrs=['_id', 'ltable._id', 'rtable._id', 'gold'], 
       target_attr='gold')

In [106]:
# Predict M 
N = rf.predict(table=M, exclude_attrs=['_id', 'ltable._id', 'rtable._id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)

In [107]:
# Evaluate the result
eval_result = mg.eval_matches(N, 'gold', 'predicted')
mg.print_eval_summary(eval_result)


Precision : 97.44% (38/39)
Recall : 97.44% (38/39)
F1 : 97.44%
False positives : 1 (out of 39 positive predictions)
False negatives : 1 (out of 96 negative predictions)


In [108]:
#Approach 2
# Apply trigger
T = neg_trigger.execute(N, 'predicted', inplace=False)

In [109]:
# Evaluate the result
eval_result = mg.eval_matches(T, 'gold', 'predicted')
mg.print_eval_summary(eval_result)

Precision : 97.37% (37/38)
Recall : 94.87% (37/39)
F1 : 96.1%
False positives : 1 (out of 38 positive predictions)
False negatives : 2 (out of 97 negative predictions)


In [135]:
#Approach 3 - Final Model used on the evaluation set J
M = mg.extract_feature_vecs(J, feature_table=feat_subset_concat, attrs_after='gold')
# Impute missing values
M.fillna(0, inplace=True)
# Train using feature vectors from I 
rf.fit(table=K_feat, 
       exclude_attrs=['_id', 'ltable._id', 'rtable._id', 'gold'], 
       target_attr='gold')
# Predict M 
N = rf.predict(table=M, exclude_attrs=['_id', 'ltable._id', 'rtable._id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)

In [138]:
# Evaluate the result
eval_result = mg.eval_matches(N, 'gold', 'predicted')
mg.print_eval_summary(eval_result)

Precision : 97.44% (38/39)
Recall : 97.44% (38/39)
F1 : 97.44%
False positives : 1 (out of 39 positive predictions)
False negatives : 1 (out of 96 negative predictions)


In [140]:
N.to_csv('final_matches_test.csv')

True

In [114]:
#Approach 4
# Apply trigger
T = neg_trigger.execute(N, 'predicted', inplace=False)

In [115]:
# Evaluate the result
eval_result = mg.eval_matches(T, 'gold', 'predicted')
mg.print_eval_summary(eval_result)

Precision : 97.37% (37/38)
Recall : 94.87% (37/39)
F1 : 96.1%
False positives : 1 (out of 38 positive predictions)
False negatives : 2 (out of 97 negative predictions)


In [109]:
N.to_csv('final_matches.csv')

True